<a href="https://colab.research.google.com/github/hamsungmin/DataTrainAnalysis/blob/main/project_week15_RAG_%EC%B2%B4%EC%9D%B8_%EA%B5%AC%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 프로세스 설계 및 데이터 로드**

<aside>

**진행 프로세스**

사내 업무 매뉴얼, 정책 문서, 회의 요약, 기술자료 등의 CSV 기반 데이터를 활용하여 사내 정보 검색 및 요약 AI 챗봇의 학습용 데이터셋을 구축합니다.
각 문서의 본문과 부가정보(작성자, 부서, 날짜, 카테고리 등)를 포함하는 CSV 파일(documents.csv)을 LangChain RAG 구조에 맞게 전처리 및 문서화합니다.

- documents.csv 파일을 불러와 행을 하나의 사내 정보 단위 문서(Document) 로 구성합니다.
    - 본문 컬럼(text, content, body 중 존재하는 항목)을 자동 탐색하여 page_content 로 지정
    - 나머지 컬럼은 metadata 로 분리하여 문서의 속성 정보(작성자, 부서, 문서유형 등)로 저장
    - 각 Document는 page_content(본문) + metadata(속성정보)의 구조로 통합
- 모든 문서를 LangChain의 Document 객체 리스트(docs)로 저장하여 RAG 파이프라인의 입력 데이터로 활용합니다.
    - 전체 Document 개수와 일부 샘플의 본문, 메타데이터를 출력하여 로드 결과 검증
</aside>

In [ ]:
import pandas as pd
from langchain.docstore.document import Document

try:
    df = pd.read_csv('/content/documents.csv', encoding='euc-kr')
    print(f"Successfully loaded documents.csv with 'euc-kr' encoding. Number of rows: {len(df)}")
except Exception as e:
    print(f"Error loading file with 'euc-kr' encoding: {e}")
    df = None


docs = []

if df is not None:
    # Identify potential text columns
    text_columns = ['text', 'content', 'body']
    found_text_column = None
    for col in text_columns:
        if col in df.columns:
            found_text_column = col
            break

    if found_text_column:
        print(f"본문 컬럼 : {found_text_column}")
        # Iterate over rows and create Document objects
        for index, row in df.iterrows():
            page_content = str(row[found_text_column])

            # Extract metadata
            metadata = row.drop(found_text_column).to_dict()

            # Create Document object
            doc = Document(page_content=page_content, metadata=metadata)
            docs.append(doc)

        print(f"총 {len(docs)} 개 행을 읽었습니다.")

        # Verify results by printing a sample
        if docs:
            print("샘플 문서:")
            print(docs[0])
        else:
            print("No documents were created.")

    else:
        print("Error: Could not find a suitable text column (text, content, or body) in the CSV.")

Successfully loaded documents.csv with 'euc-kr' encoding. Number of rows: 11
본문 컬럼 : content
총 11 개 행을 읽었습니다.
샘플 문서:
page_content='2024년 하반기 인사평가는 9월 1일부터 10월 31일까지 진행됩니다. 평가 항목은 업무성과(60%), 역량평가(30%), 동료평가(10%)로 구성됩니다. 업무성과는 상반기에 설정한 KPI 달성도를 기준으로 평가하며, S/A/B/C/D 5단계로 구분합니다. 역량평가는 리더십, 협업능력, 문제해결능력, 전문성 4개 영역에서 평가됩니다. 평가 결과는 11월 15일 개별 통보되며, 이의신청 기간은 11월 20일까지입니다. 평가 결과는 연말 성과급 및 2025년 연봉 책정에 반영됩니다.' metadata={'title': '2024년 하반기 인사평가 가이드라인', 'category': '인사', 'department': '인사팀', 'author': '김민수', 'created_date': '2024-08-15', 'updated_date': '2024-08-20', 'tags': '인사평가,KPI,성과관리,연봉', 'file_path': '/docs/hr/2024_performance_review.pdf', 'access_level': 'all'}


In [ ]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.7 MB/s eta 0:00:00


**2. 임베딩생성**

<aside>

**진행 프로세스**

각 문서의 본문과 부가정보(작성자, 부서, 작성일, 문서유형 등)를 포함한 CSV 데이터를 **Document 객체로 구조화**한 후, 이를 **청크 단위로 분할하고 벡터 인덱스로 저장**합니다. 청크 분할 방식, 임베딩 모델, 벡터DB는 데이터 특성에 맞게 적절히 선택합니다.

- RAG에 최적화된 크기로 텍스트를 청크화한 뒤, 각 청크를 **의미 벡터(embedding)** 로 변환하여 결과물을 저장합니다.
    - chunks_data.pkl : 청크 분할 결과 저장 파일
    - 전체 청크 개수, 벡터 차원, 배열 크기 등을 출력해 변환이 정상적으로 이루어졌는지 확인하고
        
        일부 샘플 벡터(예: 첫 번째 청크의 앞 10개 값)를 출력하여 임베딩 품질검증
        
- 벡터DB를 재사용할수 있도록 저장한뒤, 벡터 차원, 배열 크기 등을 출력해 변환이 정상적으로 이루어졌는지 확인합니다.
    - faiss.db (db는 사용자 선택)
</aside>

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import pickle
import os

# Define chunking parameters
chunk_size = 500
chunk_overlap = 50

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)

# Split documents into chunks
chunks = text_splitter.split_documents(docs)

print(f"총 {len(chunks)} 개의 청크로 분할되었습니다.")

# Initialize embedding model (using a Korean-friendly model)
# You might need to install the required libraries: pip install sentence-transformers
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'} # Use 'cuda' if you have a GPU
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Create embeddings and build FAISS index
# This might take some time depending on the number of chunks
print("임베딩 생성 및 FAISS 인덱스 구축 중...")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("FAISS 인덱스 구축 완료.")

# Save the FAISS index
faiss_index_path = "faiss_index"
vectorstore.save_local(faiss_index_path)
print(f"FAISS 인덱스가 '{faiss_index_path}' 경로에 저장되었습니다.")

# Optional: Save chunks data (for verification or later use)
chunks_data_path = "chunks_data.pkl"
with open(chunks_data_path, "wb") as f:
    pickle.dump(chunks, f)
print(f"청크 데이터가 '{chunks_data_path}' 경로에 저장되었습니다.")

# Verify results
print(f"전체 청크 개수: {len(chunks)}")
# Get vector dimension (assuming all vectors have the same dimension)
if chunks:
    sample_embedding = embeddings.embed_query(chunks[0].page_content)
    vector_dimension = len(sample_embedding)
    print(f"벡터 차원: {vector_dimension}")
    print(f"저장된 FAISS 인덱스 경로: {faiss_index_path}")
    print(f"저장된 청크 데이터 경로: {chunks_data_path}")

    # Print sample vector (first 10 values of the first chunk's embedding)
    print("\n샘플 벡터 (첫 번째 청크의 앞 10개 값):")
    print(sample_embedding[:10])
else:
    print("청크가 생성되지 않아 벡터 정보를 확인할 수 없습니다.")

총 11 개의 청크로 분할되었습니다.
임베딩 생성 및 FAISS 인덱스 구축 중...
FAISS 인덱스 구축 완료.
FAISS 인덱스가 'faiss_index' 경로에 저장되었습니다.
청크 데이터가 'chunks_data.pkl' 경로에 저장되었습니다.
전체 청크 개수: 11
벡터 차원: 768
저장된 FAISS 인덱스 경로: faiss_index
저장된 청크 데이터 경로: chunks_data.pkl

샘플 벡터 (첫 번째 청크의 앞 10개 값):
[-0.03705906867980957, 0.02856399491429329, -0.05304097384214401, -0.04996252804994583, 0.024541465565562248, -0.010610361583530903, 0.04500309005379677, 0.0329958014190197, 0.028273243457078934, 0.009366557002067566]


**3.  Retriever(검색기) 구현**

<aside>

**진행 프로세스**

**저장된 벡터 인덱스를 불러와**, LangChain을 이용해 **리트리버(Retriever)** 를 구성합니다.  이 리트리버는 사용자의 질문(query)을 임베딩 후, 의미적으로 유사한 문서를 검색하여 **RAG 파이프라인의 입력**으로 제공합니다.

- 저장된 로컬 벡터DB(faiss_db) 를 불러와 **LangChain의 Retriever(검색기) 로 변환**합니다.
    - as_retriever() 메서드를 사용하여 의미기반 검색이 가능하도록 구성
    - 검색 유형은 similarity로 설정하여 코사인 유사도 기반의 결과 3개의 관련문서 반환
- 각 문서의 **본문(page_content)** 과 **메타데이터(metadata)** 를 함께 검토하여 검색 결과의 정확성과 일관성을 확인합니다.
    - **메타데이터 필터링**을 통해 부서, 작성일, 문서유형 등 특정 조건에 따라 결과를 선별하여 검색 품질을 평가

**결과물**

- 질문: query = "2024년 인사평가 가이드라인에 대해 알려줘”
</aside>

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define the path to the saved FAISS index
faiss_index_path = "faiss_index"

# Initialize the same embedding model used for creating the index
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'} # Use 'cuda' if you have a GPU
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Load the FAISS index
try:
    vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    print(f"Successfully loaded FAISS index from '{faiss_index_path}'.")
except Exception as e:
    print(f"Error loading FAISS index: {e}")
    vectorstore = None


# Convert the vectorstore to a retriever
if vectorstore is not None:
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    print("FAISS vectorstore has been converted to a retriever.")

    # Define a sample query
    query = "2024년 인사평가 가이드라인에 대해 알려줘"
    print(f"Query: {query}")

    # Perform a search using the retriever
    if retriever:
        retrieved_docs = retriever.invoke(query)

        # Print the retrieved documents
        print("Retrieved Documents:")
        if retrieved_docs:
            for i, doc in enumerate(retrieved_docs):
                print(f"--- Document {i+1} ---")
                print(f"Page Content: {doc.page_content}")
                print(f"Metadata: {doc.metadata}")
                print("-" * 20)
        else:
            print("No documents were retrieved for the given query.")
    else:
        print("Retriever could not be created.")
else:
    print("Vectorstore could not be loaded, so retriever could not be created.")

Successfully loaded FAISS index from 'faiss_index'.
FAISS vectorstore has been converted to a retriever.
Query: 2024년 인사평가 가이드라인에 대해 알려줘
Retrieved Documents:
--- Document 1 ---
Page Content: 2024년 하반기 인사평가는 9월 1일부터 10월 31일까지 진행됩니다. 평가 항목은 업무성과(60%), 역량평가(30%), 동료평가(10%)로 구성됩니다. 업무성과는 상반기에 설정한 KPI 달성도를 기준으로 평가하며, S/A/B/C/D 5단계로 구분합니다. 역량평가는 리더십, 협업능력, 문제해결능력, 전문성 4개 영역에서 평가됩니다. 평가 결과는 11월 15일 개별 통보되며, 이의신청 기간은 11월 20일까지입니다. 평가 결과는 연말 성과급 및 2025년 연봉 책정에 반영됩니다.
Metadata: {'title': '2024년 하반기 인사평가 가이드라인', 'category': '인사', 'department': '인사팀', 'author': '김민수', 'created_date': '2024-08-15', 'updated_date': '2024-08-20', 'tags': '인사평가,KPI,성과관리,연봉', 'file_path': '/docs/hr/2024_performance_review.pdf', 'access_level': 'all'}
--------------------
--- Document 2 ---
Page Content: 개인정보 보호법 개정에 따라 당사의 개인정보 처리방침을 다음과 같이 개정합니다. 시행일: 2024년 11월 1일. 주요 변경사항: 1) 개인정보 수집 항목 명시 강화 - 필수 항목과 선택 항목 명확히 구분. 2) 보유 기간 구체화 - 회원정보는 회원 탈퇴 시까지, 거래정보는 5년. 3) 제3자 제공 동의 절차 강화 - 제공 목적, 항목, 보유기간 사전 고지. 4) 개인정보 열람·정정·삭제 

**4. RAG 체인 구성**

<aside>

**진행 프로세스**

사내 문서 검색 및 요약 AI 챗봇의 마지막 단계에서는 **벡터DB(faiss_db)**를 기반으로 요약형 RAG 체인을 구성하여, 사용자의 질문에 의미적으로 관련된 문서를 검색하고 핵심 요약 응답을 생성합니다.
**캐시 정책은 “반복 질의는 장기 저장, 일회성 질의는 단기 캐시**” 방식으로 설계해 효율성과 리소스 절약을 동시에 달성합니다.  또한 검색 완료 후 LLM 호출 직전에 **캐시**를 적용하여 최신 검색 결과를 유지하면서 불필요한 재계산을 방지합니다.

- 검색된 데이터의 맥락을 LLM이 이해하고 정제된 답변으로 변환하는 역할을 수행하여, 사내문서 내 핵심정보를 요약형 자연어 응답으로 생성합니다.
    - ChatPromptTemplate 을 사용해 “사용자 질문 + 검색된 문서 내용(context)”을 하나의 입력으로 통합하는 프롬프트 구조를 정의
- **메모리 캐시 전략(Cache Strategy)** 을 적용하여 검색·요약 성능과 응답 속도를 최적화합니다.
    - **InMemoryCache** 를 통해 동일한 질의 반복 시 결과를 즉시 재사용하도록 구성하여 불필요한 LLM 호출을 최소화
    - **SQLiteCache** 나 파일 기반 캐시를 추가 적용하면, 세션이 종료되어도 과거 검색·요약 결과를 재활용
    - 캐시 정책은 “**자주 반복되는 사내 질의(FAQ, 절차 안내 등)** 는 장기 저장 / 일회성 질의는 세션 단기 캐시” 방식으로 설계하여, 처리 효율성과 리소스 절약을 동시에 확보합니다.
    - 캐시 저장 및 갱신 시점은 **검색 완료 후 LLM 호출 직전**에 배치하여, 최신 검색 결과를 유지하면서도 불필요한 재계산을 방지합니다.
</aside>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# from langchain_community.llms import Ollama # 주석 처리 또는 삭제
from langchain_openai import ChatOpenAI # OpenAI Chat 모델 사용
from langchain.globals import set_llm_cache
from langchain_community.cache import InMemoryCache
import os
from google.colab import userdata # 코랩 Secrets에서 API 키 가져오기

# Set up caching
set_llm_cache(InMemoryCache())
print("InMemoryCache가 설정되었습니다.")

# Define the prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
print("ChatPromptTemplate가 정의되었습니다.")

# Initialize the language model (using OpenAI API)
# Get API key from Colab Secrets
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    if not openai_api_key:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets.")

    # Use ChatOpenAI for chat models
    llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key) # 또는 "gpt-3.5-turbo" 등 다른 모델 사용
    print(f"OpenAI 모델 '{llm.model_name}' 이(가) 초기화되었습니다.")

except Exception as e:
    print(f"OpenAI 모델 초기화 오류: {e}")
    llm = None
    print("코랩 Secrets에 'OPENAI_API_KEY'를 올바르게 설정했는지 확인하세요.")


# Create the RAG chain
if llm is not None and 'retriever' in locals():
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    print("RAG 체인이 구성되었습니다.")

    # Define a sample query
    query = "2024년 인사평가 가이드라인에 대해 알려줘"
    print(f"\nQuery: {query}")

    # Invoke the RAG chain
    print("RAG 체인 호출 중...")
    try:
        response = rag_chain.invoke(query)
        print("\n응답:")
        print(response)
    except Exception as e:
        print(f"RAG 체인 호출 오류: {e}")

else:
    if llm is None:
        print("LLM (OpenAI)이 초기화되지 않아 RAG 체인을 구성할 수 없습니다.")
    if 'retriever' not in locals():
        print("Retriever가 정의되지 않아 RAG 체인을 구성할 수 없습니다. 이전 단계를 먼저 완료해주세요.")

InMemoryCache가 설정되었습니다.
ChatPromptTemplate가 정의되었습니다.
OpenAI 모델 'gpt-4o-mini' 이(가) 초기화되었습니다.
RAG 체인이 구성되었습니다.

Query: 2024년 인사평가 가이드라인에 대해 알려줘
RAG 체인 호출 중...

응답:
2024년 하반기 인사평가는 9월 1일부터 10월 31일까지 진행됩니다. 평가 항목은 다음과 같이 구성됩니다:
- 업무성과: 60% (상반기에 설정한 KPI 달성도를 기준으로 평가)
- 역량평가: 30% (리더십, 협업능력, 문제해결능력, 전문성 4개 영역에서 평가)
- 동료평가: 10%

평가는 S/A/B/C/D 5단계로 구분되며, 평가 결과는 11월 15일에 개별 통보됩니다. 이의신청 기간은 11월 20일까지입니다. 평가 결과는 연말 성과급 및 2025년 연봉 책정에 반영됩니다.


In [ ]:
%pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.6 MB/s eta 0:00:00
